In [1]:
import pandas as pd

# Load the CSV files
tariff_file_path = 'tariff_pred.csv'
solar_file_path = 'solar_madeupdata.csv'

tariff_data = pd.read_csv(tariff_file_path)
solar_data = pd.read_csv(solar_file_path)

# Ensure the datetime columns are in datetime format
tariff_data['datetime'] = pd.to_datetime(tariff_data['datetime'])
solar_data['datetime'] = pd.to_datetime(solar_data['datetime'], format='%d-%m-%Y %H:%M')

# Define the hourly pattern for devices (except dishwasher, washing machine, and pump)
def get_power_usage(hour):
    if 18 <= hour <= 21:   # 6pm to 10pm (maximum usage)
        return 5  # kW
    elif 22 <= hour <= 23:  # Slow drop until 12am
        return 3.5  # kW
    elif 0 <= hour <= 4:    # Constant usage till 5am
        return 2  # kW
    elif 5 <= hour <= 8:    # Slow rise till 9am
        return 3  # kW
    elif 9 <= hour <= 12:   # Slight drop till 1pm
        return 2.5  # kW
    elif 13 <= hour <= 15:  # Slow rise till 4pm
        return 3  # kW
    elif 16 <= hour <= 17:  # Slight drop till 6pm
        return 2.5  # kW
    else:
        return 1.5  # Default fallback

# Set date range from July 1, 2024 to October 20, 2024
start_date = pd.to_datetime("2024-07-01 00:00:00")
end_date = pd.to_datetime("2024-10-20 23:59:59")
date_range = pd.date_range(start=start_date, end=end_date, freq='H')

# Initialize power usage data
power_usage = [get_power_usage(dt.hour) for dt in date_range]
usage_data = pd.DataFrame({
    'datetime': date_range,
    'power_usage_kW': power_usage
})

# Merge the usage_data with solar_data to only keep matching rows
merged_data = pd.merge(usage_data, solar_data[['datetime', 'ac_power_output_kW']], on='datetime', how='inner')

# Initialize battery parameters
battery_capacity_kWh = 30  # Example battery capacity
battery_level = 0.5 * battery_capacity_kWh  # 50% initial charge
battery_levels = []

for i, row in merged_data.iterrows():
    current_hour = row['datetime'].hour
    
    # Get solar output for the current hour
    solar_output = row['ac_power_output_kW']
    
    # Get the tariff for the current hour
    tariff = tariff_data.loc[tariff_data['datetime'] == row['datetime'], 'tariff_pred']
    tariff = tariff.values[0] if not tariff.empty else float('inf')  # Default to inf if no tariff data
    
    # Battery management logic
    if 23 < current_hour or current_hour < 5:  # Nighttime, no grid charging, no battery use
        battery_discharge = 0
    else:
        if solar_output > 0:
            # Charge the battery with solar output
            battery_level = min(battery_level + solar_output, battery_capacity_kWh)
            battery_discharge = 0  # No battery discharge when charging
        elif tariff <= 4 and battery_level < battery_capacity_kWh:  # Low tariff, charge from the grid
            battery_level = min(battery_level + 1, battery_capacity_kWh)
            battery_discharge = 0  # No discharge while charging from the grid
        else:
            # Discharge the battery if needed
            battery_discharge = min(row['power_usage_kW'], battery_level)
            battery_level -= battery_discharge

    battery_levels.append(battery_level)

# Add the battery levels to the DataFrame
merged_data['battery_level_kWh'] = battery_levels

# Save the results to a CSV file without an index
merged_data.to_csv('consumption_battery_data.csv', index=False)

# Display some of the results
print(merged_data.head())


C:\Users\DELL\AppData\Local\Temp\ipykernel_99308\3106431977.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
C:\Users\DELL\AppData\Local\Temp\ipykernel_99308\3106431977.py:36: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start=start_date, end=end_date, freq='H')


             datetime  power_usage_kW  ac_power_output_kW  battery_level_kWh
0 2024-07-01 00:00:00             2.0                 0.0               15.0
1 2024-07-01 01:00:00             2.0                 0.0               15.0
2 2024-07-01 02:00:00             2.0                 0.0               15.0
3 2024-07-01 03:00:00             2.0                 0.0               15.0
4 2024-07-01 04:00:00             2.0                 0.0               15.0
